In [27]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [28]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from facebook_scraper import get_posts, get_profile
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import datetime
from ast import literal_eval
import numpy as np
import csv
import os
import re

Account, Facebook login password and FanPage ID we need to Crawl

In [29]:
EMAIL = "thanhdat3108k67@gmail.com"
PASSWORD = "22022627VNU"
PAGE_ID = "schannel.vn"

In [30]:
def initDriver():
    """
    Initialize the Chrome driver with options to disable notifications.
    This effectively creates a new browser window.
    """
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-notifications")
    return webdriver.Chrome(options=options)

def fbLogin(driver, email, password):
    """
    Log in to Facebook using the provided email and password after 
    the browser window is opened.
    """
    driver.get("https://facebook.com")

    # Login process
    userEmail = driver.find_element(by="id", value="email")
    userEmail.send_keys(email)
    userPass = driver.find_element(by="id", value="pass")
    userPass.send_keys(password)
    userPass.send_keys(Keys.ENTER)

## Let's continue to scratch all missing reactions and comments with selenium

In [31]:
# This was written few days later
# I found out that the facebook_scraper library is not stable
# So let's load up our data and strip data we won't need, fill in the missing data with different method
df = pd.read_csv('Data/raw_schannel.csv')
df.head(5)

# Drop unnecessary columns
df = df[['post_id','text', 'time', 'comments', 'shares', 'comments_full']]
# TODO:
# Add reactions count for each post
# Add comments text for each post
df.head(5)

#NaN value type
df.dropna(subset=['comments'], inplace=True)
df.dropna(subset=['post_id'], inplace=True)



In [32]:
import pandas as pd
from datetime import datetime
df['time'] = pd.to_datetime(df['time'])

## Function Crawls all reactions

In [33]:
def get_count_react(href):
        ''' 
        Get reaction type and count from the given href
        '''
        reaction_type = re.search(r'reaction_type=(\d+)', href).group(1)
        total_count = re.search(r'total_count=(\d+)', href).group(1)
        if reaction_type == '1':
            reaction_type = 'like'
        elif reaction_type == '2':
                reaction_type = 'love'
        elif reaction_type == '16':
               reaction_type = 'care'
        elif reaction_type == '4':
               reaction_type = 'haha'
        elif reaction_type == '3':
                reaction_type = 'wow'
        elif reaction_type == '7':
                reaction_type = 'sad'
        else:
                reaction_type = 'angry'
        return reaction_type, int(total_count)

In [34]:
def get_reactions(driver, post_id):
    '''
    Get all reactions of a post
    '''
    driver.get(f"https://mbasic.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier={post_id}")
    sleep(10)
    links = driver.find_elements(By.XPATH, value="//a[@role='button' and @aria-pressed='false']")
    
    hrefs = [link.get_attribute('href') for link in links]
    reactions = {}
    for href in hrefs:
        reaction_type, count = get_count_react(href)
        reactions[reaction_type] = count
    return reactions

## Funtion Crawls comment_full

In [35]:
def get_comments_per_site(driver):
    '''
    Get all comments appearing on site max 10
    {
        'commenter_name': name,
        'commenter_id': commenter_id,
        'text': text,
        'comment_id': comment_id,
    }
    '''
    comments = []
    
    boxes = driver.find_elements(By.XPATH, value="//html/body/div/div/div[2]/div/div/div[2]/div/div[5]/*")
    
    for box in boxes:
        try:
            commenter_name = box.find_element(By.XPATH, value=".//h3/a").text
            def get_id_from_href(href):
                id_match = re.search(r'\?id=(\d+)', href)
                if id_match:
                    return id_match.group(1)
                else:
                    return re.search(r'/([^?]+)', href).group(1)
            commenter_id =  get_id_from_href(box.find_element(By.XPATH, value=".//h3/a").get_attribute('href'))
            comment_text = box.find_element(By.XPATH, value=".//div/div").text
            comment_id = box.get_attribute('id')
            comments.append({
                'commenter_name': commenter_name,
                'commenter_id': commenter_id,
                'text': comment_text,
                'comment_id': comment_id,
            })
        except: # Have see previous
            continue

    return comments

In [36]:
def get_comment_full(driver, amount, post_id):
    ''' 
    Get <amount> comments of a post
    '''
    driver.get(f"https://mbasic.facebook.com/{post_id}")
    sleep(10)

    comments = []
    while True:
        comments += get_comments_per_site(driver)
        if len(comments) >= amount:
            break
        sleep(20)
    
        # Find see_next else break
        try:
            button = driver.find_element(By.XPATH, value="//div[contains(@id, 'see_next')]")
            button.find_element(By.XPATH, value=".//a").click()
            sleep(10)
        except:
            break
    return comments[:amount]

In [37]:
driver = initDriver()
fbLogin(driver, EMAIL, PASSWORD)
sleep(5)

In [38]:
df['reactions'] = df['post_id'].apply(lambda x: get_reactions(driver, x))
df['reaction_count'] = df['reactions'].apply(lambda x: sum(x.values()))
df['comments_full'] = df['post_id'].apply(lambda x: get_comment_full(driver, 20, x)) # Get 20 comments for each post

KeyboardInterrupt: 

## Export file 



In [40]:
df.to_csv('Data/schannel.csv', index = False)
df.to_excel('Data/schannel.xlsx', index = False)